# User Routes

> a functional approach, routes only, to interacting with domo users.


In [ ]:
# | default_exp routes.user


In [ ]:
# | exporti
import utils.DictDot as dd
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


# GET Users

In [ ]:
# | export
async def get_all_users(
    auth: dmda.DomoAuth, debug_api: bool = False
) -> rgd.ResponseGetData:

    """retrieves all users from Domo"""
    url = f"https://{ auth.domo_instance}.domo.com/api/content/v2/users"

    return await gd.get_data(url=url, method="GET", auth=auth, debug_api=debug_api)

#### sample implementation of get_all_users


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_all_users(auth=token_auth)
pd.DataFrame(res.response[0:5])

os.environ['DOMO_DOJO_ACCESS_TOKEN']


'2f4cb212853732ef796abd46077cf1162c0a3ed7cf1ff079'

# Search Users
This series of functions searchse the v1_users_api.  The body of the search api can be retrieved by searching admin in the Domo User interface.

In [ ]:
# | export
def generate_search_users_body_by_id(
    user_ids: list[str],  # list of user ids to search
) -> dict:  # dict to pass to search v1_users_search_api
    """search v1_users_search_api"""

    return {
        # "showCount": true,
        # "count": false,
        "includeDeleted": False,
        "includeSupport": False,
        "filters": [
            {"field": "id", "filterType": "value", "values": user_ids, "operator": "EQ"}
        ],
    }

In [ ]:
# | export
def generate_search_users_body_by_email(
    user_email_ls: list[ str ],  # list of user emails to search.  Note:  search does not appear to be case sensitive
) -> dict:  # dict to pass to search v1_users_search_api
    """search v1_users_search_api"""

    return {
        # "showCount": true,
        # "count": false,
        "includeDeleted": False,
        "includeSupport": False,
        "limit": 200,
        "offset": 0,
        "sort": {"field": "displayName", "order": "ASC"},
        "filters": [
            {
                "filterType": "text",
                "field": "emailAddress",
                "text": " ".join(user_email_ls),
            }
        ],
    }

In [ ]:
os.environ['DOMO_DOJO_ACCESS_TOKEN']

'2f4cb212853732ef796abd46077cf1162c0a3ed7cf1ff079'

In [ ]:
# | export
def process_v1_search_users(v1_user_ls: list[dict] # list of users from v1_users_search API
) -> list[dict]: # sanitized list of users.
    """sanitizes the response from v1_users_search API and removes unecessary attributes"""

    clean_users = []

    for obj_user in v1_user_ls:
        
        dd_user = dd.DictDot(obj_user)

        clean_users.append( {
            "id": dd_user.id,
            "displayName": dd_user.displayName,
            "roleId": dd_user.roleId,
            "userName": dd_user.userName,
            "emailAddress": dd_user.emailAddress,
        })
    
    return clean_users


In [ ]:
# | export
async def search_users(
    auth: dmda.DomoAuth,
    body: dict,
    debug_api: bool = False,
    process_users: bool = True,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/users/search"

    res = await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

    if process_users and res.is_success:
        res.response = process_v1_search_users(res.response.get('users'))

    return res


#### sample implementation of search_users


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

search_email_body = generate_search_users_body_by_email(
    user_email_ls=["jae@onyxreporting.com"]
)

res = await search_users(
    auth=token_auth, 
    body=search_email_body, 
    process_users=True, 
    debug_api=False
)

pd.DataFrame(res.response)

,id,displayName,roleId,userName,emailAddress
0,1893952720,Jae Wilson,1,jae@onyxreporting.com,jae@onyxreporting.com


# Publish and Virtual Users
> Virtual Users are necessary for assigning PDP policies when publish jobs are created.

In [ ]:
# | export
async def search_virtual_user_by_subscriber_instance(
    auth: dmda.DomoAuth, # domo auth object
    subscriber_instance_ls : list[str], # list of subscriber domo instances
    debug_api: bool = False, # debug API requests
) -> rgd.ResponseGetData: # list of virtual domo users
    """retrieve virtual users for subscriber instances tied to one publisher"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/proxy_user/domain/"

    body = {"domains": [f"{subscriber_instance}.domo.com"
                        for subscriber_instance in subscriber_instance_ls]}

    return await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

#### sample of search virtual users by subscriber_instance

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await search_virtual_user_by_subscriber_instance(
    auth = token_auth,
    subscriber_instance_ls = ['domo-dojo', 'test'],
    debug_api = False
    )


pd.DataFrame(res.response)

,id,publisherDomain,customerId,subscriberDomain,virtualUserId,created,activeSubscriptions
0,2bcabab7-128e-4f47-8219-c8548588d7fc,domo-dojo.domo.com,mmmm-0012-0200,domo-dojo.domo.com,fc:ce69e5f0-ab88-46df-94ab-32b5f6dfd4e7,1674251102000,0
1,57999e9b-e8a2-48ab-80c6-edf2627b71fc,domo-dojo.domo.com,mmmm-0012-0200,test.domo.com,fc:b3d7aaa1-9bc8-4481-9d34-c153a9b91b22,1674253476000,0


# CRUD Routes and User Attributes


In [ ]:
# | export
async def create_user(
    auth: dmda.DomoAuth,
    display_name: str,
    email: str,
    role_id: int,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/users"

    body = {"displayName": display_name, "detail": {"email": email}, "roleId": role_id}

    return await gd.get_data(
        url=url, method="POST", body=body, auth=auth, debug_api=debug_api
    )

In [ ]:
# | export
async def set_user_landing_page(
    auth: dmda.DomoAuth, user_id: str, page_id: str, debug_api: bool = False
):

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/landings/target/DESKTOP/entity/PAGE/id/{page_id}/{user_id}"

    return await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        # body = body,
        debug_api=debug_api,
    )

In [ ]:
# | export
async def reset_password(
    auth: dmda.DomoAuth,
    user_id: str,
    new_password: str,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/password"

    body = {"domoUserId": user_id, "password": new_password}

    return await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

In [ ]:
# | export
async def request_password_reset(
    domo_instance: str, email: str, locale="en-us", debug_api: bool = False
):
    url = f"https://{domo_instance}.domo.com/api/domoweb/auth/sendReset"

    params = {"email": email, "local": locale}

    return await gd.get_data(
        url=url, method="GET", params=params, auth=None, debug_api=debug_api
    )

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
